In [23]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

In [24]:
from scripts.dataloader import DataLoader
from scripts.data_understanding import DataUnderstanding
from scripts.visualization import Visualization

In [25]:
dl = DataLoader('../data', 'user_engagment_data.csv')
user_engagment = dl.read_csv()
user_engagment = user_engagment.sort_values(by=['MSISDN/Number'], ascending=True)

In [26]:
dl = DataLoader('../data', 'user_experiance_data.csv')
user_experiance = dl.read_csv()
user_experiance = user_experiance.sort_values(by=['MSISDN/Number'], ascending=True)

In [27]:
user_engagment.head()

,session_traffic(bytes),session_duration(ms),session_freq,cluster,engagment_score,MSISDN/Number
24229,8.786906e+08,116720.0,1,2,0.001961,3.360100e+10
99191,1.568596e+08,181230.0,1,1,0.000938,3.360100e+10
55784,5.959665e+08,134969.0,1,2,0.001867,3.360100e+10
73840,4.223207e+08,49878.0,1,2,0.001975,3.360101e+10
7414,1.457411e+09,37104.0,2,2,0.002068,3.360101e+10


In [28]:
user_experiance.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,experiance_score
41943,3.360100e+10,46.000000,0.0,37.0,39.000000,2.080991e+07,759658.664811,Huawei P20 Lite Huawei Nova 3E,0.000581
127709,3.360100e+10,30.000000,1.0,48.0,51.000000,2.080991e+07,759658.664811,Apple iPhone 7 (A1778),0.000581
134590,3.360100e+10,109.795706,5.0,48.0,49.000000,2.080991e+07,759658.664811,undefined,0.000581
95551,3.360101e+10,69.000000,15.0,204.0,44.000000,1.066000e+03,759658.664811,Apple iPhone 5S (A1457),0.000042
16589,3.360101e+10,86.000000,5.0,40372.0,1770.428647,9.349630e+06,21202.000000,Apple iPhone Se (A1723),0.000239


### Merge the User experiance and User engagment table

In [29]:
import pandas as pd
user_satsfaction = pd.merge(user_engagment, user_experiance, on="MSISDN/Number")

In [32]:
user_satsfaction.head()

,session_traffic(bytes),session_duration(ms),session_freq,cluster,engagment_score,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,experiance_score,avg_satsfaction_score
0,8.786906e+08,116720.0,1,2,0.001961,3.360100e+10,46.000000,0.0,37.0,39.000000,2.080991e+07,759658.664811,Huawei P20 Lite Huawei Nova 3E,0.000581,0.001271
1,1.568596e+08,181230.0,1,1,0.000938,3.360100e+10,30.000000,1.0,48.0,51.000000,2.080991e+07,759658.664811,Apple iPhone 7 (A1778),0.000581,0.000759
2,5.959665e+08,134969.0,1,2,0.001867,3.360100e+10,109.795706,5.0,48.0,49.000000,2.080991e+07,759658.664811,undefined,0.000581,0.001224
3,4.223207e+08,49878.0,1,2,0.001975,3.360101e+10,69.000000,15.0,204.0,44.000000,1.066000e+03,759658.664811,Apple iPhone 5S (A1457),0.000042,0.001009
4,1.457411e+09,37104.0,2,2,0.002068,3.360101e+10,86.000000,5.0,40372.0,1770.428647,9.349630e+06,21202.000000,Apple iPhone Se (A1723),0.000239,0.001154


### The average of both engagement & experience scores

In [31]:
user_satsfaction['avg_satsfaction_score'] = (user_satsfaction['engagment_score'] + user_satsfaction['experiance_score'])/2

### Prepare the data for the regression

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [41]:
X = user_satsfaction.loc[:, user_satsfaction.columns!='Handset Type']
X = X.iloc[:, :-1]
Y = user_satsfaction.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

### Regression

In [42]:
model = LinearRegression()

In [43]:
model = model.fit(x_train, y_train)

In [44]:
prediction = model.predict(x_test)

In [45]:
prediction

array([0.00123705, 0.00125827, 0.00126193, ..., 0.0012292 , 0.00120729,
       0.00096184])

### K-means (k=2) on the engagement & the experience score 

In [49]:
from sklearn.cluster import KMeans
kmeans = KMeans(init="random",
                n_clusters=2,
                n_init=10,
                max_iter=300,
                random_state=42
   )

In [50]:
from scripts.datacleaning import DataCleaning
cleaner = DataCleaning()
normalized = cleaner.normalizer(X)
normalized

array([[2.61417834e-02, 3.47251814e-06, 2.97508408e-11, ...,
        6.19112447e-04, 2.26004840e-05, 1.72734507e-14],
       [4.66825081e-03, 5.39352939e-06, 2.97606875e-11, ...,
        6.19317355e-04, 2.26079641e-05, 1.72791676e-14],
       [1.77337759e-02, 4.01618224e-06, 2.97563310e-11, ...,
        6.19226697e-04, 2.26046547e-05, 1.72766378e-14],
       ...,
       [7.26063355e-05, 2.74438307e-07, 3.12791200e-13, ...,
        6.50915806e-06, 2.37614546e-07, 1.03690378e-17],
       [1.76939988e-06, 7.50830778e-10, 2.96735872e-15, ...,
        6.17504807e-08, 2.25417977e-09, 1.17269510e-19],
       [1.57699572e-07, 9.85773856e-10, 1.13327661e-15, ...,
        2.35833890e-08, 8.60903393e-10, 4.48297219e-20]])

In [51]:
clustering_model = kmeans.fit(normalized)

In [53]:
centroids = kmeans.cluster_centers_
centroids

array([[9.98061863e-01, 1.50500754e-04, 2.00606671e-09, 3.70123009e-12,
        3.59491929e-15, 6.22293749e-02, 1.44082315e-10, 1.78427126e-11,
        1.89684201e-08, 1.37256886e-09, 1.58896912e-05, 6.88955593e-07,
        4.61665605e-16],
       [2.72716710e-02, 6.32870129e-06, 5.50990474e-11, 5.45262945e-11,
        5.40577586e-14, 9.99402031e-01, 1.91772109e-09, 2.39534501e-10,
        2.35193047e-07, 1.68124713e-08, 4.14677224e-04, 1.59822691e-05,
        1.16434889e-14]])

In [54]:
user_satsfaction['statsfaction_cluster'] = model.predict(normalized)

/home/hmicheal/.conda/envs/10x/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


### Persistent the data to mysql database

In [63]:
from sqlalchemy import create_engine
import os
import pymysql

In [64]:
from dotenv import load_dotenv
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"), echo=True)
conn = engine.connect()

2022-08-27 10:33:33,494 INFO sqlalchemy.engine.Engine SELECT DATABASE()


2022-08-27 10:33:33,494:INFO:sqlalchemy.engine.Engine:SELECT DATABASE()


2022-08-27 10:33:33,495 INFO sqlalchemy.engine.Engine [raw sql] {}


2022-08-27 10:33:33,495:INFO:sqlalchemy.engine.Engine:[raw sql] {}


2022-08-27 10:33:33,497 INFO sqlalchemy.engine.Engine SELECT @@sql_mode


...

2022-08-27 10:33:33,497:INFO:sqlalchemy.engine.Engine:[raw sql] {}


2022-08-27 10:33:33,498 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names


2022-08-27 10:33:33,498:INFO:sqlalchemy.engine.Engine:SELECT @@lower_case_table_names


2022-08-27 10:33:33,499 INFO sqlalchemy.engine.Engine [raw sql] {}


2022-08-27 10:33:33,499:INFO:sqlalchemy.engine.Engine:[raw sql] {}


In [65]:
user_satsfaction.to_sql('user_satsfaction',con = engine)

2022-08-27 10:34:29,424 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s


2022-08-27 10:34:29,424:INFO:sqlalchemy.engine.Engine:SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s


2022-08-27 10:34:29,425 INFO sqlalchemy.engine.Engine [generated in 0.00128s] {'table_schema': 'telecomdata', 'table_name': 'user_satsfaction'}


2022-08-27 10:34:29,425:INFO:sqlalchemy.engine.Engine:[generated in 0.00128s] {'table_schema': 'telecomdata', 'table_name': 'user_satsfaction'}


2022-08-27 10:34:29,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)


...

2022-08-27 10:34:30,972 INFO sqlalchemy.engine.Engine [generated in 1.27184s] ({'index': 0, 'session_trafficAbytesZ': 878690574.0, 'session_durationAmsZ': 116720.0, 'session_freq': 1, 'cluster': 2, 'engagment_score': 0.0019605127842471, 'MSISDN/Number': 33601001722.0, 'Avg RTT DL AmsZ': 46.0, 'Avg RTT UL AmsZ': 0.0, 'Avg Bearer TP DL AkbpsZ': 37.0, 'Avg Bearer TP UL AkbpsZ': 39.0, 'TCP DL Retrans. Vol ABytesZ': 20809914.27034193, 'TCP UL Retrans. Vol ABytesZ': 759658.6648110661, 'Handset Type': 'Huawei P20 Lite Huawei Nova 3E', 'experiance_score': 0.0005806037811166, 'avg_satsfaction_score': 0.0012705582826818501, 'statsfaction_cluster': 0.0004978991983963694}, {'index': 1, 'session_trafficAbytesZ': 156859643.0, 'session_durationAmsZ': 181230.0, 'session_freq': 1, 'cluster': 1, 'engagment_score': 0.0009379837598242, 'MSISDN/Number': 33601001754.0, 'Avg RTT DL AmsZ': 30.0, 'Avg RTT UL AmsZ': 1.0, 'Avg Bearer TP DL AkbpsZ': 48.0, 'Avg Bearer TP UL AkbpsZ': 51.0, 'TCP DL Retrans. Vol AByt

2022-08-27 10:34:30,972:INFO:sqlalchemy.engine.Engine:[generated in 1.27184s] ({'index': 0, 'session_trafficAbytesZ': 878690574.0, 'session_durationAmsZ': 116720.0, 'session_freq': 1, 'cluster': 2, 'engagment_score': 0.0019605127842471, 'MSISDN/Number': 33601001722.0, 'Avg RTT DL AmsZ': 46.0, 'Avg RTT UL AmsZ': 0.0, 'Avg Bearer TP DL AkbpsZ': 37.0, 'Avg Bearer TP UL AkbpsZ': 39.0, 'TCP DL Retrans. Vol ABytesZ': 20809914.27034193, 'TCP UL Retrans. Vol ABytesZ': 759658.6648110661, 'Handset Type': 'Huawei P20 Lite Huawei Nova 3E', 'experiance_score': 0.0005806037811166, 'avg_satsfaction_score': 0.0012705582826818501, 'statsfaction_cluster': 0.0004978991983963694}, {'index': 1, 'session_trafficAbytesZ': 156859643.0, 'session_durationAmsZ': 181230.0, 'session_freq': 1, 'cluster': 1, 'engagment_score': 0.0009379837598242, 'MSISDN/Number': 33601001754.0, 'Avg RTT DL AmsZ': 30.0, 'Avg RTT UL AmsZ': 1.0, 'Avg Bearer TP DL AkbpsZ': 48.0, 'Avg Bearer TP UL AkbpsZ': 51.0, 'TCP DL Retrans. Vol AByt

2022-08-27 10:34:36,244 INFO sqlalchemy.engine.Engine COMMIT


2022-08-27 10:34:36,244:INFO:sqlalchemy.engine.Engine:COMMIT


150001